#Data Loading

In [1]:
#importing packages
import pandas as pd
from google.colab import files
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
#loading data
complete = pd.read_csv("complete_merged_data.csv")

In [ ]:
complete_df = complete.copy()

#FEATURE ENGINEERING

In [ ]:
#Feature 1 : Store age in days
complete_df['Open_Date'] = pd.to_datetime(complete_df['Open_Date'], format='%Y-%m-%d', errors='coerce')
complete_df['Store_Age_Days'] = (datetime.now() - complete_df['Open_Date']).dt.days

In [ ]:
#Feature 2: High_Education_High_Income: Interaction between Perc_Population_Bachelor_Degree and Perc_HH_Income_above100K
complete_df['High_Education_High_Income'] = ( complete_df['Perc_Population_Bachelor_Degree'] * complete_df['Perc_HH_Income_above100K'])

In [ ]:
#Feature 3: Diversity_Index: Sum of Perc_Hispanic, Perc_Asian, and Perc_African_American
complete_df['Diversity_Index'] = ( complete_df['Perc_Hispanic'] + complete_df['Perc_Asian'] + complete_df['Perc_African_American'] )

In [ ]:
#Feature 4:  Age_Income_Ratio: Ratio of Perc_Population_Age_50-70 to Median_HH_Income
complete_df['Age_Income_Ratio'] = ( complete_df['Perc_Population_Age_50-70'] / complete_df['Median_HH_Income'] )

In [ ]:
#Feature 5: Price_Per_Household: Ratio of Retail_Price to Households
complete_df['Price_Per_Household'] = ( complete_df['Retail_Price'] / complete_df['Households'])

In [ ]:
#Feature 6:  Vodka_Sales_Factor: Product of Vodka and Retail_Price
complete_df['Vodka_Sales_Factor'] = ( complete_df['Vodka'] * complete_df['Retail_Price'])

In [ ]:
#Feature 7: # Household_Income_to_NetWorth_Ratio: Ratio of Median_HH_Income to Average_Net_Worth
complete_df['Household_Income_to_NetWorth_Ratio'] = ( complete_df['Median_HH_Income'] / complete_df['Average_Net_Worth'] )

In [ ]:
#Feature 8: Household and Vodka sales interaction: combined factor of household size and product sales
complete_df['Household_Vodka_Factor'] = complete_df['Households'] * complete_df['Vodka']

In [ ]:
#Feature 9: Education and diversity interaction: capturing the diversity effect among educated populations
complete_df['Education_Diversity_Factor'] = complete_df['Perc_Population_Bachelor_Degree'] * complete_df['Diversity_Index']

In [ ]:
#Feature 10: Population diversity and vodka interaction: measuring diversity effect in vodka consumption
complete_df['Diversity_Vodka_Factor'] = complete_df['Diversity_Index'] * complete_df['Vodka']

In [ ]:
#Feature 11: Population age and high-income interaction: older population in high-income areas
complete_df['Age_HighIncome_Factor'] = complete_df['Perc_Population_Age_50-70'] * complete_df['Perc_HH_Income_above100K']

In [ ]:
#Feature 12: One-hot Encoding Store_Size
complete_df = pd.get_dummies(complete_df, columns=['Store_Size'], prefix='Store_Size')

In [ ]:
#Feature 13: Wealth Diversity
complete_df['Wealth_Diversity_Index'] = complete_df['Average_Net_Worth'] * complete_df['Diversity_Index']

In [ ]:
#Feature 14: Store_State dummies
complete_df = pd.get_dummies(complete_df, columns=['Store_State'], prefix='Store_State')

In [ ]:
#Feature 15: Package_Type
complete_df = pd.get_dummies(complete_df, columns=['Package_Type'], prefix='Package_Type')

In [ ]:
#Cleaning NA's
complete_df['Count_Week_Instock'] = complete_df.groupby(['Item_Code', 'Market_Name'])['Count_Week_Instock'].transform(
    lambda x: x.fillna(x.median())
)

# Check if the NAs have been replaced successfully
print(complete_df['Count_Week_Instock'].isna().sum())
#fill rest with median
complete_df['Count_Week_Instock'].fillna(complete_df['Count_Week_Instock'].median(), inplace=True)

# Check again if all NAs have been replaced
print(complete_df['Count_Week_Instock'].isna().sum())

11
0


<ipython-input-20-d19f813672c1>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  complete_df['Count_Week_Instock'].fillna(complete_df['Count_Week_Instock'].median(), inplace=True)


In [ ]:
#Feature 16: Count_Week_Instock_Normalized
scaler = MinMaxScaler()
complete_df['Count_Week_Instock_Normalized'] = scaler.fit_transform(complete_df[['Count_Week_Instock']])

In [ ]:
complete_df.columns

Index(['Store_Number', 'Item_Code', 'Item_Name', 'Retail_Price',
       'Count_Week_Instock', 'Normalized_Sales_$L52W', 'Sales Bucket',
       'Store_Name', 'Open_Date', 'Market_Name', 'Store_Address', 'Households',
       'Perc_HH_Income_above100K', 'Median_HH_Income', 'Average_Net_Worth',
       'Perc_Population_Bachelor_Degree', 'Perc_Hispanic', 'Perc_Asian',
       'Perc_African_American', 'Perc_Population_Age_50-70', 'US Whiskey',
       'Tequila Under $65', 'Tequila Over $65', 'Scotch Under $75',
       'Scotch Over $75', 'Vodka', 'Cordials', 'Brandy Under $85',
       'Brandy Over $85', 'Cabernet Under $20', 'Cabernet $20-50',
       'Cabernet Over $50', 'Chardonnay Under $20', 'Chardonnay Over $20',
       'Wine - Sparkling', 'Pinot Noir Under $20', 'Pinot Noir Over $20',
       'Sauvignon Blanc', 'French Champagne', 'Market_Sales_L52wk',
       'Count_Item_Location', 'Store_Age_Days', 'High_Education_High_Income',
       'Diversity_Index', 'Age_Income_Ratio', 'Price_Per_Househ

In [ ]:
complete_df = complete_df.replace({True: 1, False: 0})

<ipython-input-23-558b02ee8866>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  complete_df = complete_df.replace({True: 1, False: 0})


In [ ]:
complete_df.to_csv('complete_df.csv', index=False)
files.download('complete_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>